# Load Libraries 

In [2]:
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D, Flatten, GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import regularizers

Using TensorFlow backend.


In [0]:
from hyperopt import hp, tpe, fmin

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [0]:
import dask.dataframe as dd

In [0]:
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

# Set-up

In [0]:
RANDOM_SEED = 314 #used to help randomly select the data points
TEST_PCT = 0.2 # 20% of the data
rcParams['figure.figsize'] = 14, 8.7 # Golden Mean
LABELS = ["Normal","Fraud"]
col_list = ["cerulean","scarlet"]# https://xkcd.com/color/rgb/
#sns.set(style='white', font_scale=1.75, palette=sns.xkcd_palette(col_list))

# Load Data 

In [0]:
data_file_path = "train.csv"
data = dd.read_csv(data_file_path)
data_df = data.compute()

In [8]:
data_df.shape

(891, 12)

In [0]:
del data,data_file_path

In [10]:
data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
X = data_df.drop(['Survived'], axis=1)
Y = data_df.Survived

In [0]:
X.drop(columns=['Name','Sex','Ticket','Cabin','Embarked'],inplace=True)

# Normalize Data

In [80]:
d.Age.fillna(d.Age.median(),inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [82]:
d.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [0]:
d = data_df[data_df.columns[~data_df.columns.isin(['Name','Sex','Ticket','Cabin','Embarked'])]]

In [88]:
data_df[data_df.columns[~data_df.columns.isin(['Name','Sex','Ticket','Cabin','Embarked'])]].head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500


In [0]:
d_ = []
for i in range(d.shape[0]):
  integer_ = d.loc[i,['PassengerId','Pclass','SibSp']].values
  float_   = d.loc[i,['Age','Fare','Parch']].values
  a = np.vstack((integer_,float_))
  a = a.tolist()
  d_.append(a)
d_ = np.array(d_)
d_ = np.reshape(d_,(d_.shape[0],2,3,1))    

In [90]:
d_.shape

(891, 2, 3, 1)

In [91]:
print(d.shape)
d.Survived.value_counts()

(891, 7)


0    549
1    342
Name: Survived, dtype: int64

In [0]:
y_train_onehot = pd.get_dummies(d['Survived']).values

In [95]:
model = Sequential()
model.add(Convolution2D(100, (1, 1),
                        padding='valid',
                        input_shape=(2, 3, 1)))
model.add(Activation('relu'))

model.add(Convolution2D(100, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 2, 3, 100)         200       
_________________________________________________________________
activation_79 (Activation)   (None, 2, 3, 100)         0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 2, 3, 100)         10100     
_________________________________________________________________
activation_80 (Activation)   (None, 2, 3, 100)         0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 1, 1, 100)         0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 1, 1, 100)         0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 100)             

In [96]:
model.fit(d_, y_train_onehot, epochs=5, validation_split=0.1, batch_size=20)

Train on 801 samples, validate on 90 samples
Epoch 1/5
801/801 [==============================] - 1s 2ms/step - loss: nan - acc: 0.6167 - val_loss: nan - val_acc: 0.6222
Epoch 2/5
801/801 [==============================] - 0s 151us/step - loss: nan - acc: 0.6155 - val_loss: nan - val_acc: 0.6222
Epoch 3/5
801/801 [==============================] - 0s 140us/step - loss: nan - acc: 0.6155 - val_loss: nan - val_acc: 0.6222
Epoch 4/5
801/801 [==============================] - 0s 136us/step - loss: nan - acc: 0.6155 - val_loss: nan - val_acc: 0.6222
Epoch 5/5
801/801 [==============================] - 0s 156us/step - loss: nan - acc: 0.6155 - val_loss: nan - val_acc: 0.6222


# Split Data

# Search Space 

In [0]:
IMG_WIDTH = IMG_HEIGHT = 224

In [0]:
search_space = {
    'pooling': hp.choice('pooling', ['AVG', 'MAX', 'FLATTEN']),
    'first_layers': hp.choice('first_layers',[256, 512, 1024]),
    'hidden_layers': hp.choice('hidden_layers',[256, 512, 1024]),
    'activation':hp.choice('activation', ['elu', 'relu']),
    'dropout':hp.choice('dropout', [0, 0.25, 0.5]),
    'opt_1':hp.choice('opt_1', ['adam', 'sgd', 'rmsprop', 'nadam']),
    'opt_2':hp.choice('opt_2', ['adam', 'sgd', 'rmsprop', 'nadam']),
    'kernel_initializer':hp.choice('kernel_initializer', ['lecun_uniform', 'lecun_normal']),
    'num_hidden_layers':hp.choice('num_hidden_layers', [1, 2, 3]),
    'num_filters':hp.choice('num_filters', [64, 128, 256]),
    'num_convolutions':hp.choice('num_convolutions', [2, 3, 4]),
    'id_model':hp.uniform('id_model', 0, 1)
}